In [ ]:
"""


"""


import pandas as pd
import random
from statistics import mode

POP_SIZE = 50
PATTERN_SIZE = 5
GENERATIONS = 100

 
url = 'https://raw.githubusercontent.com/cardosource/previsao-jogos/main/2021.csv'
data = pd.read_csv(url, header=None) 
data = data.values.tolist()

 
def find_most_frequent_pattern(population):
    pattern_counts = {}
    for candidate in population:
        for i in range(len(candidate) - PATTERN_SIZE + 1):
            pattern = tuple(candidate[i:i+PATTERN_SIZE])
            pattern_counts[pattern] = pattern_counts.get(pattern, 0) + 1
    sorted_patterns = sorted(pattern_counts.items(), key=lambda x: x[1], reverse=True)
    most_frequent_patterns = set()
    for pattern, count in sorted_patterns:
        if len(most_frequent_patterns) == 6:
            break
        if pattern[0] not in most_frequent_patterns:
            most_frequent_patterns.add(pattern[0])
    return most_frequent_patterns

def fitness(candidate, data):
    pattern_counts = {}
    for row in candidate:
        for i in range(len(row) - PATTERN_SIZE + 1):
            pattern = tuple(row[i:i+PATTERN_SIZE])
            pattern_counts[pattern] = pattern_counts.get(pattern, 0) + 1
    most_frequent_pattern = mode(pattern_counts)
    return pattern_counts[most_frequent_pattern]


 
def selection(population):
    weights = [fitness(candidate, data) for candidate in population]
    total_weight = sum(weights)
    if total_weight == 0: 
        return random.choices(population, k=2)
    normalized_weights = [w / total_weight for w in weights]
    return random.choices(population, weights=normalized_weights, k=2)

 def crossover(parents):
    cut_point = random.randrange(1, PATTERN_SIZE - 1)
    offspring1 = parents[0][:cut_point] + parents[1][cut_point:]
    offspring2 = parents[1][:cut_point] + parents[0][cut_point:]
    return offspring1, offspring2

 def mutation(candidate):
    for row in candidate:
        indices = random.sample(range(len(row)), 2)
        row[indices[0]], row[indices[1]] = row[indices[1]], row[indices[0]]
    return candidate

 population = [[random.choices(range(1, 61), k=60) for _ in range(6)] for _ in range(POP_SIZE)]

# evolui a população
for generation in range(GENERATIONS):
    parents = selection(population)  
    offspring1, offspring2 = crossover(parents)
    offspring1 = mutation(offspring1)
    offspring2 = mutation(offspring2)
    new_population = [offspring1, offspring2]

     best1, best2 = sorted(population, key=lambda candidate: fitness(candidate, data=data), reverse=True)[:2]
    
    new_population += [best1, best2]

    while len(new_population) < POP_SIZE:
        parents = selection(population)
        offspring1, offspring2 = crossover(parents)
        offspring1 = mutation(offspring1)
        offspring2 = mutation(offspring2)
        if offspring1 not in new_population:
            new_population.append(offspring1)
        if offspring2 not in new_population:
            new_population.append(offspring2)
     
    # atualiza a população
    population = new_population
    
    print(f"Geracao {generation+1} - padrão dominante na população : {fitness(population[0], data=data)}")
    best_candidate = max(population, key=lambda candidate: fitness(candidate, data))
    if fitness(best_candidate, data) == len(data):
        break


ix=0
for row in data:
    pattern_counts = {}
    for i in range(len(row) - PATTERN_SIZE + 1):
        pattern = tuple(row[i:i+PATTERN_SIZE])
        pattern_counts[pattern] = pattern_counts.get(pattern, 0) + 1
    most_frequent_pattern = max(pattern_counts, key=pattern_counts.get)
    new_sequence = list(most_frequent_pattern)[:6] * 2
    new_sequence = new_sequence[:6]

    print(f'{ix} - {new_sequence}')
    ix+=1